In [5]:
import pandas as pd
import geopandas as gpd
import numpy as np
import pickle
import copy
import geodata
import time
import datetime as dt
from datetime import date, datetime, timedelta
from functions import calc_easter
from tqdm import tqdm
import math

In [6]:
# GLOBAL SETTINGS
xl_engine = "openpyxl"
#input_path_base = pdir_folder
ref_w_year = 2015
ref_c_year = 2015
leap_year = "cutdec31"

# YEARS
years_list = [2030]
utc = True

In [7]:
# Set the number of clusters!
clusters = 10

In [8]:
print("Read region-specific ev load profiles and shift potential.")
dict_profiles = pd.read_pickle("data/X_nuts_profiles.pickle")
dict_neg_flex = pd.read_pickle("data/X_nuts_neg_flex.pickle")
dict_pos_flex = pd.read_pickle("data/X_nuts_pos_flex.pickle")
dict_driving = pd.read_pickle("data/X_nuts_driving.pickle")

Read region-specific ev load profiles and shift potential.


In [9]:
print("Read Siedlungstyp-specific ev load profiles and shift potential.")
dict_profiles_Siedlungstyp = pd.read_pickle("data/X_siedlungstyp_profiles.pickle")
dict_neg_flex_Siedlungstyp = pd.read_pickle("data/X_siedlungstyp_neg_flex.pickle")
dict_pos_flex_Siedlungstyp = pd.read_pickle("data/X_siedlungstyp_pos_flex.pickle")
dict_driving_Siedlungstyp = pd.read_pickle("data/X_siedlungstyp_driving.pickle")

Read Siedlungstyp-specific ev load profiles and shift potential.


In [10]:
# get regions
nutsList = list(dict_profiles)
nutsList[:5]

['DEB25', 'DE138', 'DEA2D', 'DE11D', 'DE71D']

In [11]:
SiedlungstypList = list(dict_profiles_Siedlungstyp.keys())
SiedlungstypList

['Landgemeinde',
 'Größere Kleinstadt',
 'Großstadt',
 'Mittelstadt',
 'Kleine Kleinstadt']

In [12]:
# change 2020 to 2019, as 2019 is the key for the other data
if 2020 in years_list:
    for nested_d in dict_profiles.values():
        nested_d[2019] = nested_d.pop(2020)
    for nested_d in dict_neg_flex.values():
        nested_d[2019] = nested_d.pop(2020)
    for nested_d in dict_pos_flex.values():
        nested_d[2019] = nested_d.pop(2020)
    for nested_d in dict_driving.values():
        nested_d[2019] = nested_d.pop(2020)
    # do the same for Siedlungstypen
    for nested_d in dict_profiles_Siedlungstyp.values():
        nested_d[2019] = nested_d.pop(2020)
    for nested_d in dict_neg_flex_Siedlungstyp.values():
        nested_d[2019] = nested_d.pop(2020)
    for nested_d in dict_pos_flex_Siedlungstyp.values():
        nested_d[2019] = nested_d.pop(2020)
    for nested_d in dict_driving_Siedlungstyp.values():
        nested_d[2019] = nested_d.pop(2020)

In [13]:
# Step 1: Generate yearly charging profiles per region and charging type
# Generate dataframe for calendar year:
leap_years = [1980, 1984, 1988, 1992, 1996, 2000, 2004, 2008, 2012, 2016, 2020]
      
# this is a day longer than necessary so that mez time can be cut out later
utc_w = pd.date_range(start="01/01/"+str(ref_w_year), end="01/02/"+str(ref_w_year+1), freq="H")[:-1].strftime('%Y-%m-%d %H:%M:%S')
utc_c = pd.date_range(start="01/01/"+str(ref_c_year), end="01/02/"+str(ref_c_year+1), freq="H")[:-1].strftime('%Y-%m-%d %H:%M:%S')

if ref_w_year in leap_years:
    if leap_year == "cutdec31":
        utc_w = utc_w[~utc_w.str.contains("12-31")]
    elif leap_year == "cutfeb29":
        utc_w = utc_w[~utc_w.str.contains("02-29")]
    
if ref_c_year in leap_years:
    if leap_year == "cutdec31":
        utc_c = utc_c[~utc_c.str.contains("12-31")]
    elif leap_year == "cutfeb29":
        utc_c = utc_c[~utc_c.str.contains("02-29")]

In [14]:
# time is mez
mez_c = pd.to_datetime(pd.to_datetime(utc_c)+timedelta(days=0, hours=1)).strftime('%Y-%m-%d %H:%M:%S')
DayTable = pd.DataFrame(index=sorted(list(set([d[:-9] for d in mez_c])))[:-1])
if ref_c_year in leap_years:
    if leap_year == "cutdec31":
        DayTable = DayTable[~DayTable.index.str.contains("12-31")]
    elif leap_year == "cutfeb29":
        DayTable = DayTable[~DayTable.index.str.contains("02-29")]

date_df = pd.DataFrame(index=pd.to_datetime(DayTable.index))  # MEZ dates
date_df["Wochentag"] = ["Werktag" if i <= 4 else "Samstag" if i == 5 else "Sonntag" for i in date_df.index.weekday]
holidays = pd.read_excel("data/Feiertage.xlsx")
holidates = [dt.date(ref_c_year, int(holidays.loc[i,"Monat"]), int(holidays.loc[i,"Tag"])) for i in holidays[["Tag", "Monat"]].dropna().index]
holidates += [dt.date(ref_c_year-1, 12, 31), dt.date(ref_c_year+1, 1, 1)]
easter = calc_easter(ref_c_year)
holidates += [easter+timedelta(holidays['Tage nach Ostersonntag'].loc[i]) for i in holidays[['Tage nach Ostersonntag']].dropna().index]
date_df["Wochentag"] = [date_df["Wochentag"].loc[i] if not i in holidates else "Sonntag" for i in date_df.index]
date_df["season"] = ["So-" if (i>=date(ref_c_year,5,15) and i<=date(ref_c_year,9,14)) else "Wi-" for i in date_df.index]
date_df["Profiltag"] = [date_df["season"].loc[i]+date_df["Wochentag"].loc[i] for i in date_df.index]

FileNotFoundError: [Errno 2] No such file or directory: 'data/Feiertage.xlsx'

In [ ]:
# Check if there are columns missing
for reg in nutsList:
    for y in years_list:
        for d in list(['Wochentag', 'Wochenende']):
            for i in range(clusters):
                if i not in dict_pos_flex[reg][y][d]:
                    print(" not existing: pos_flex", reg, y, d, i)
                    dict_pos_flex[reg][y][d][i] = 0 
                if i not in dict_neg_flex[reg][y][d]:
                    print(" not existing: neg_flex", reg, y, d, i)
                    dict_neg_flex[reg][y][d][i] = 0
                if i not in dict_profiles[reg][y][d]:
                    print(" not existing: profile", reg, y, d, i)
                    dict_profiles[reg][y][d][i] = 0
                if i not in dict_driving[reg][y][d]:
                    print(" not existing: driving", reg, y, d, i)
                    dict_driving[reg][y][d][i] = 0

# Calculating Profiles for variable number of clusters
Way down below you can also find the old code, where the cluster size was hardcoded to 8. I only slightly changed that code.

Directly following is the code without hardcoding that should work for every "clusters" value. I changed much more in this code compared to arnoldfs original code.

In [15]:
"""
Calculate Profiles for Siedlungstypen
"""

dict_profiles_daily_Siedlungstyp = {}
dict_pos_flex_daily_Siedlungstyp = {}
dict_neg_flex_daily_Siedlungstyp = {}
dict_driving_daily_Siedlungstyp = {}
print("Generation of yearly charging profiles for each Siedlungstyp")
for reg in tqdm(SiedlungstypList):
    dict_profiles_daily_Siedlungstyp[reg] = {}
    dict_pos_flex_daily_Siedlungstyp[reg] = {}
    dict_neg_flex_daily_Siedlungstyp[reg] = {}
    dict_driving_daily_Siedlungstyp[reg] = {}
    for y in years_list:
        profiles_Siedlungstyp = {d: pd.DataFrame({"time": pd.date_range(start=d, periods=24, freq="h"), **{i: dict_profiles_Siedlungstyp[reg][y]['Wochentag'][i] for i in range(clusters)}
                                     }) if date_df["Wochentag"].loc[d] == "Werktag"
                                 else pd.DataFrame({"time": pd.date_range(start=d, periods=24, freq="h"), **{i: dict_profiles_Siedlungstyp[reg][y]['Wochenende'][i] for i in range(clusters)}}) for d in date_df.index}
        pos_flex_Siedlungstyp = {d: pd.DataFrame({"time": pd.date_range(start=d, periods=24, freq="h"), **{i: dict_pos_flex_Siedlungstyp[reg][y]['Wochentag'][i] for i in range(clusters)}
                                     }) if date_df["Wochentag"].loc[d] == "Werktag"
                                 else pd.DataFrame({"time": pd.date_range(start=d, periods=24, freq="h"), **{i: dict_pos_flex_Siedlungstyp[reg][y]['Wochenende'][i] for i in range(clusters)}}) for d in date_df.index}
        neg_flex_Siedlungstyp = {d: pd.DataFrame({"time": pd.date_range(start=d, periods=24, freq="h"), **{i: dict_neg_flex_Siedlungstyp[reg][y]['Wochentag'][i] for i in range(clusters)}
                                     }) if date_df["Wochentag"].loc[d] == "Werktag"
                                 else pd.DataFrame({"time": pd.date_range(start=d, periods=24, freq="h"), **{i: dict_neg_flex_Siedlungstyp[reg][y]['Wochenende'][i] for i in range(clusters)}}) for d in date_df.index}
        driving_Siedlungstyp = {d: pd.DataFrame({"time": pd.date_range(start=d, periods=24, freq="h"), **{i: dict_driving_Siedlungstyp[reg][y]['Wochentag'][i] for i in range(clusters)}
                                     }) if date_df["Wochentag"].loc[d] == "Werktag"
                                 else pd.DataFrame({"time": pd.date_range(start=d, periods=24, freq="h"), **{i: dict_driving_Siedlungstyp[reg][y]['Wochenende'][i] for i in range(clusters)}}) for d in date_df.index}

        profiles_Siedlungstyp_df = pd.DataFrame(columns=["time"] + list(range(clusters)))
        pos_flex_Siedlungstyp_df = pd.DataFrame(columns=["time"] + list(range(clusters)))
        neg_flex_Siedlungstyp_df = pd.DataFrame(columns=["time"] + list(range(clusters)))
        driving_Siedlungstyp_df = pd.DataFrame(columns=["time"] + list(range(clusters)))

        for d in pos_flex_Siedlungstyp:
            profiles_Siedlungstyp_df = pd.concat([profiles_Siedlungstyp_df, profiles_Siedlungstyp[d]])
            pos_flex_Siedlungstyp_df = pd.concat([pos_flex_Siedlungstyp_df, pos_flex_Siedlungstyp[d]])
            neg_flex_Siedlungstyp_df = pd.concat([neg_flex_Siedlungstyp_df, neg_flex_Siedlungstyp[d]])
            driving_Siedlungstyp_df = pd.concat([driving_Siedlungstyp_df, driving_Siedlungstyp[d]])

        profiles_Siedlungstyp_df = profiles_Siedlungstyp_df.set_index("time", drop=True)
        pos_flex_Siedlungstyp_df = pos_flex_Siedlungstyp_df.set_index("time", drop=True)
        neg_flex_Siedlungstyp_df = neg_flex_Siedlungstyp_df.set_index("time", drop=True)
        driving_Siedlungstyp_df = driving_Siedlungstyp_df.set_index("time", drop=True)

        profiles_Siedlungstyp_df["utc_c"] = pd.to_datetime(pd.to_datetime(profiles_Siedlungstyp_df.index)).strftime('%Y-%m-%d %H:%M:%S')
        pos_flex_Siedlungstyp_df["utc_c"] = pd.to_datetime(pd.to_datetime(pos_flex_Siedlungstyp_df.index)).strftime('%Y-%m-%d %H:%M:%S')
        neg_flex_Siedlungstyp_df["utc_c"] = pd.to_datetime(pd.to_datetime(neg_flex_Siedlungstyp_df.index)).strftime('%Y-%m-%d %H:%M:%S')
        driving_Siedlungstyp_df["utc_c"] = pd.to_datetime(pd.to_datetime(driving_Siedlungstyp_df.index)).strftime('%Y-%m-%d %H:%M:%S')


        if utc:
            profiles_Siedlungstyp_df = profiles_Siedlungstyp_df[profiles_Siedlungstyp_df.utc_c.str.contains(str(ref_c_year))]
            profiles_Siedlungstyp_df = profiles_Siedlungstyp_df.set_index("utc_c")
            
            pos_flex_Siedlungstyp_df = pos_flex_Siedlungstyp_df[pos_flex_Siedlungstyp_df.utc_c.str.contains(str(ref_c_year))]
            pos_flex_Siedlungstyp_df = pos_flex_Siedlungstyp_df.set_index("utc_c")
            
            neg_flex_Siedlungstyp_df = neg_flex_Siedlungstyp_df[neg_flex_Siedlungstyp_df.utc_c.str.contains(str(ref_c_year))]
            neg_flex_Siedlungstyp_df = neg_flex_Siedlungstyp_df.set_index("utc_c")
            
            driving_Siedlungstyp_df = driving_Siedlungstyp_df[driving_Siedlungstyp_df.utc_c.str.contains(str(ref_c_year))]
            driving_Siedlungstyp_df = driving_Siedlungstyp_df.set_index("utc_c")

        else:
            profiles_Siedlungstyp_df.index = profiles_Siedlungstyp_df.index.strftime('%Y-%m-%d %H:%M:%S')
            profiles_Siedlungstyp_df = profiles_Siedlungstyp_df[profiles_Siedlungstyp_df.index.str.contains(str(ref_c_year))]
            profiles_Siedlungstyp_df = profiles_Siedlungstyp_df.drop(["utc_c"], axis=1)

            pos_flex_Siedlungstyp_df.index = pos_flex_Siedlungstyp_df.index.strftime('%Y-%m-%d %H:%M:%S')
            pos_flex_Siedlungstyp_df = pos_flex_Siedlungstyp_df[pos_flex_Siedlungstyp_df.index.str.contains(str(ref_c_year))]
            pos_flex_Siedlungstyp_df = pos_flex_Siedlungstyp_df.drop(["utc_c"], axis=1)
            
            neg_flex_Siedlungstyp_df.index = neg_flex_Siedlungstyp_df.index.strftime('%Y-%m-%d %H:%M:%S')
            neg_flex_Siedlungstyp_df = neg_flex_Siedlungstyp_df[neg_flex_Siedlungstyp_df.index.str.contains(str(ref_c_year))]
            neg_flex_Siedlungstyp_df = neg_flex_Siedlungstyp_df.drop(["utc_c"], axis=1)
            
            driving_Siedlungstyp_df.index = driving_Siedlungstyp_df.index.strftime('%Y-%m-%d %H:%M:%S')
            driving_Siedlungstyp_df = driving_Siedlungstyp_df[driving_Siedlungstyp_df.index.str.contains(str(ref_c_year))]
            driving_Siedlungstyp_df = driving_Siedlungstyp_df.drop(["utc_c"], axis=1)

        dict_profiles_daily_Siedlungstyp[reg][y] = profiles_Siedlungstyp_df
        dict_pos_flex_daily_Siedlungstyp[reg][y] = pos_flex_Siedlungstyp_df
        dict_neg_flex_daily_Siedlungstyp[reg][y] = neg_flex_Siedlungstyp_df
        dict_driving_daily_Siedlungstyp[reg][y] = driving_Siedlungstyp_df
                          

# Save results
print("Save profiles to pickle")
with open("data/profiles_daily_Siedlungstyp_clusters" + str(clusters) + ".pickle", 'wb') as file:
    pickle.dump(dict_profiles_daily_Siedlungstyp, file, protocol=pickle.HIGHEST_PROTOCOL)

print("Save pos flex potential to pickle")
with open("data/pos_flex_daily_Siedlungstyp_clusters" + str(clusters) + ".pickle", 'wb') as file:
    pickle.dump(dict_pos_flex_daily_Siedlungstyp, file, protocol=pickle.HIGHEST_PROTOCOL)

print("Save neg flex potential to pickle")
with open("data/neg_flex_daily_Siedlungstyp_clusters" + str(clusters) + ".pickle", 'wb') as file:
    pickle.dump(dict_neg_flex_daily_Siedlungstyp, file, protocol=pickle.HIGHEST_PROTOCOL)

print("Save driving to pickle")
with open("data/driving_daily_Siedlungstyp_clusters" + str(clusters) + ".pickle", 'wb') as file:
    pickle.dump(dict_driving_daily_Siedlungstyp, file, protocol=pickle.HIGHEST_PROTOCOL)

Generation of yearly charging profiles for each Siedlungstyp


100%|██████████| 5/5 [00:17<00:00,  3.58s/it]

Save profiles to pickle
Save pos flex potential to pickle
Save neg flex potential to pickle
Save driving to pickle


In [16]:
print(sum(dict_profiles_daily_Siedlungstyp["Großstadt"][2030].loc[:,7]))
print(sum(dict_driving_daily_Siedlungstyp["Großstadt"][2030].loc[:,7]))

458871.89244046854
458871.8924404718


In [17]:
"""
Calculate Profiles for NUTS3
"""

dict_profiles_daily = {}
dict_pos_flex_daily = {}
dict_neg_flex_daily = {}
dict_driving_daily = {}
print("Generation of yearly charging profiles for each NUTS region")

for reg in tqdm(nutsList):
    dict_profiles_daily[reg] = {}
    dict_pos_flex_daily[reg] = {}
    dict_neg_flex_daily[reg] = {}
    dict_driving_daily[reg] = {}
    for y in years_list:
        profiles = {d: pd.DataFrame({"time": pd.date_range(start=d, periods=24, freq="h"), **{i: dict_profiles[reg][y]['Wochentag'][i] for i in range(clusters)}
                                     }) if date_df["Wochentag"].loc[d] == "Werktag"
                    else pd.DataFrame({"time": pd.date_range(start=d, periods=24, freq="h"), **{i: dict_profiles[reg][y]['Wochenende'][i] for i in range(clusters)}}) for d in date_df.index}
        
        pos_flex = {d: pd.DataFrame({"time": pd.date_range(start=d, periods=24, freq="h"), **{i: dict_pos_flex[reg][y]['Wochentag'][i] for i in range(clusters)}
                                     }) if date_df["Wochentag"].loc[d] == "Werktag"
                    else pd.DataFrame({"time": pd.date_range(start=d, periods=24, freq="h"), **{i: dict_pos_flex[reg][y]['Wochenende'][i] for i in range(clusters)}}) for d in date_df.index}
                    
        neg_flex = {d: pd.DataFrame({"time": pd.date_range(start=d, periods=24, freq="h"), **{i: dict_neg_flex[reg][y]['Wochentag'][i] for i in range(clusters)}
                                     }) if date_df["Wochentag"].loc[d] == "Werktag"
                    else pd.DataFrame({"time": pd.date_range(start=d, periods=24, freq="h"), **{i: dict_neg_flex[reg][y]['Wochenende'][i] for i in range(clusters)}}) for d in date_df.index}

        driving = {d: pd.DataFrame({"time": pd.date_range(start=d, periods=24, freq="h"), **{i: dict_driving[reg][y]['Wochentag'][i] for i in range(clusters)}
                                     }) if date_df["Wochentag"].loc[d] == "Werktag"
                    else pd.DataFrame({"time": pd.date_range(start=d, periods=24, freq="h"), **{i: dict_driving[reg][y]['Wochenende'][i] for i in range(clusters)}}) for d in date_df.index}

        
        profiles_df = pd.DataFrame(columns=["time"] + list(range(clusters)))
        pos_flex_df = pd.DataFrame(columns=["time"] + list(range(clusters)))
        neg_flex_df = pd.DataFrame(columns=["time"] + list(range(clusters)))
        driving_df = pd.DataFrame(columns=["time"] + list(range(clusters)))
        
        for d in pos_flex:
            profiles_df = pd.concat([profiles_df, profiles[d]])
            pos_flex_df = pd.concat([pos_flex_df, pos_flex[d]])
            neg_flex_df = pd.concat([neg_flex_df, neg_flex[d]])
            driving_df = pd.concat([driving_df, driving[d]])

        profiles_df = profiles_df.set_index("time", drop=True)
        pos_flex_df = pos_flex_df.set_index("time", drop=True)
        neg_flex_df = neg_flex_df.set_index("time", drop=True)
        driving_df = driving_df.set_index("time", drop=True)

        profiles_df["utc_c"] = pd.to_datetime(pd.to_datetime(profiles_df.index)).strftime(
            '%Y-%m-%d %H:%M:%S')
        pos_flex_df["utc_c"] = pd.to_datetime(pd.to_datetime(pos_flex_df.index)).strftime(
            '%Y-%m-%d %H:%M:%S')
        neg_flex_df["utc_c"] = pd.to_datetime(pd.to_datetime(neg_flex_df.index)).strftime(
            '%Y-%m-%d %H:%M:%S')
        driving_df["utc_c"] = pd.to_datetime(pd.to_datetime(driving_df.index)).strftime(
            '%Y-%m-%d %H:%M:%S')

        if utc:

            profiles_df = profiles_df[profiles_df.utc_c.str.contains(str(ref_c_year))]
            profiles_df = profiles_df.set_index("utc_c")

            pos_flex_df = pos_flex_df[pos_flex_df.utc_c.str.contains(str(ref_c_year))]
            pos_flex_df = pos_flex_df.set_index("utc_c")

            neg_flex_df = neg_flex_df[neg_flex_df.utc_c.str.contains(str(ref_c_year))]
            neg_flex_df = neg_flex_df.set_index("utc_c")
            
            driving_df = driving_df[driving_df.utc_c.str.contains(str(ref_c_year))]
            driving_df = driving_df.set_index("utc_c")

        else:
            profiles_df.index = profiles_df.index.strftime('%Y-%m-%d %H:%M:%S')
            profiles_df = profiles_df[profiles_df.index.str.contains(str(ref_c_year))]
            profiles_df = profiles_df.drop(["utc_c"], axis=1)

            pos_flex_df.index = pos_flex_df.index.strftime('%Y-%m-%d %H:%M:%S')
            pos_flex_df = pos_flex_df[pos_flex_df.index.str.contains(str(ref_c_year))]
            pos_flex_df = pos_flex_df.drop(["utc_c"], axis=1)

            neg_flex_df.index = neg_flex_df.index.strftime('%Y-%m-%d %H:%M:%S')
            neg_flex_df = neg_flex_df[neg_flex_df.index.str.contains(str(ref_c_year))]
            neg_flex_df = neg_flex_df.drop(["utc_c"], axis=1)
            
            driving_df.index = driving_df.index.strftime('%Y-%m-%d %H:%M:%S')
            driving_df = driving_df[driving_df.index.str.contains(str(ref_c_year))]
            driving_df = driving_df.drop(["utc_c"], axis=1)

        dict_profiles_daily[reg][y] = profiles_df
        dict_pos_flex_daily[reg][y] = pos_flex_df
        dict_neg_flex_daily[reg][y] = neg_flex_df
        dict_driving_daily[reg][y] = driving_df
   

# Save results
print("Save profiles to pickle")
with open("data/profiles_daily_clusters" + str(clusters) + ".pickle", 'wb') as file:
    pickle.dump(dict_profiles_daily, file, protocol=pickle.HIGHEST_PROTOCOL)

print("Save pos flex potential to pickle")
with open("data/pos_flex_daily_clusters" + str(clusters) + ".pickle", 'wb') as file:
    pickle.dump(dict_pos_flex_daily, file, protocol=pickle.HIGHEST_PROTOCOL)

print("Save neg flex potential to pickle")
with open("data/neg_flex_daily_clusters" + str(clusters) + ".pickle", 'wb') as file:
    pickle.dump(dict_neg_flex_daily, file, protocol=pickle.HIGHEST_PROTOCOL)
    
print("Save driving to pickle")
with open("data/driving_daily_clusters" + str(clusters) + ".pickle", 'wb') as file:
    pickle.dump(dict_driving_daily, file, protocol=pickle.HIGHEST_PROTOCOL)

Generation of yearly charging profiles for each NUTS region


 47%|████▋     | 184/394 [10:37<12:07,  3.47s/it]


KeyError: 2

In [ ]:
print(sum(dict_profiles_daily["DEB25"][2030].loc[:,1]))
print(sum(dict_driving_daily["DEB25"][2030].loc[:,1]))